In [1]:
import os
from pprint import pprint

from loguru import logger
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from tqdm import tqdm
import yaml

from triagerx.model.lbt_p_deberta import LBTPDeberta
from triagerx.loss.loss_functions import *
from triagerx.system.triagerx import TriagerX
from triagerx.dataset.text_processor import TextProcessor

tqdm.pandas()

In [2]:
developer_config_path = "/home/mdafifal.mamun/notebooks/triagerX/training/training_config/developer/developer_summary_sp_tokens.yaml"
component_config_path = "/home/mdafifal.mamun/notebooks/triagerX/training/training_config/component/component_summary_sp_tokens.yaml"

with open(developer_config_path, "r") as stream:
    developer_config = yaml.safe_load(stream)

with open(component_config_path, "r") as stream:
    component_config = yaml.safe_load(stream)

# Load Data

In [3]:
dataset_path = "/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/df_all_summarized.csv"
test_size = developer_config["test_size"]
seed = 42

torch.manual_seed(seed)

In [4]:
raw_df = pd.read_csv(dataset_path)
    
logger.debug(f"Raw dataset size: {len(raw_df)}")

raw_df = raw_df.rename(columns={"assignees": "owner", "issue_body": "description"})

df = TextProcessor.prepare_dataframe(
    raw_df,
    use_special_tokens=developer_config["use_special_tokens"],
    use_summary=developer_config["use_summary"],
    use_description=developer_config["use_description"],
    component_training=False,
)

df = df.sort_values(by="issue_number")
df = df[df["owner"].notna()]
# df = df[df["component"].notna()]

num_issues = len(df)
logger.info(f"Total number of issues after processing: {num_issues}")

# Define active user map
vm_users = [
    "pshipton",
    "keithc-ca",
    "gacholio",
    "tajila",
    "babsingh",
    "JasonFengJ9",
    "fengxue-IS",
    "hangshao0",
    "theresa.t.mammarella",
    "ChengJin01",
    "singh264",
    "thallium",
    "ThanHenderson",
]
jvmti_users = ["gacholio", "tajila", "babsingh", "fengxue-IS"]
jclextensions_users = ["JasonFengJ9", "pshipton", "keithc-ca"]
test_users = ["LongyuZhang", "annaibm", "sophiaxu0424", "KapilPowar", "llxia"]
build_users = ["adambrousseau", "mahdipub"]
gc_users = ["dmitripivkine", "amicic", "kangyining", "LinHu2016"]

# Putting them in dictionaries
components = {
    "comp:vm": vm_users,
    "comp:jvmti": jvmti_users,
    "comp:jclextensions": jclextensions_users,
    "comp:test": test_users,
    "comp:build": build_users,
    "comp:gc": gc_users,
}

expected_users = [user for user_list in components.values() for user in user_list]
df = df[df["owner"].isin(expected_users)]
logger.info(f"Total issues after developer filtering: {len(df)}")

df = df.sort_values(by="issue_number")

df_train, df_test = train_test_split(df, test_size=test_size, shuffle=False)

sample_threshold = 10
developers = df_train["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
df_train = df_train[df_train["owner"].isin(filtered_developers)]

train_owners = set(df_train["owner"])
test_owners = set(df_test["owner"])

unwanted = list(test_owners - train_owners)

df_test = df_test[~df_test["owner"].isin(unwanted)]

logger.info(f"Training data: {len(df_train)}, Validation data: {len(df_test)}")
logger.info(f"Number of developers: {len(df_train.owner.unique())}")

logger.info(f"Train dataset size: {len(df_train)}")
logger.info(f"Test dataset size: {len(df_test)}")

2024-05-16 13:32:11.003 | DEBUG    | __main__:<module>:3 - Raw dataset size: 7758
100%|██████████| 7758/7758 [00:00<00:00, 1072209.39it/s]
2024-05-16 13:32:11.025 | INFO     | triagerx.dataset.text_processor:prepare_dataframe:64 - Adding summary...
100%|██████████| 7758/7758 [00:00<00:00, 119526.48it/s]
2024-05-16 13:32:11.382 | INFO     | triagerx.dataset.text_processor:prepare_dataframe:71 - Adding description...
100%|██████████| 7758/7758 [00:00<00:00, 99480.91it/s]
2024-05-16 13:32:11.472 | INFO     | __main__:<module>:20 - Total number of issues after processing: 4381
2024-05-16 13:32:11.475 | INFO     | __main__:<module>:56 - Total issues after developer filtering: 1992
2024-05-16 13:32:11.480 | INFO     | __main__:<module>:74 - Training data: 1577, Validation data: 378
2024-05-16 13:32:11.482 | INFO     | __main__:<module>:75 - Number of developers: 15
2024-05-16 13:32:11.483 | INFO     | __main__:<module>:77 - Train dataset size: 1577
2024-05-16 13:32:11.485 | INFO     | __main

### Developer Mapping

In [5]:
# # Generate component ids
lbl2idx = {}

train_owners = sorted(train_owners)

for idx, dev in enumerate(train_owners):
    lbl2idx[dev] = idx

df_train["owner_id"] = df_train["owner"].apply(lambda owner: lbl2idx[owner])
df_test["owner_id"] = df_test["owner"].apply(lambda owner: lbl2idx[owner])

assert set(df_train.owner.unique()) == set(df_test.owner.unique())

In [6]:
developer_id_map = dict(zip(df_train["owner"], df_train["owner_id"]))
print(developer_id_map)

{'tajila': 13, 'hangshao0': 9, 'keithc-ca': 10, 'ChengJin01': 0, 'fengxue-IS': 7, 'llxia': 11, 'babsingh': 5, 'amicic': 4, 'pshipton': 12, 'JasonFengJ9': 1, 'dmitripivkine': 6, 'gacholio': 8, 'LinHu2016': 2, 'LongyuZhang': 3, 'thallium': 14}


In [7]:
component_id_map = {key: index for index, key in enumerate(sorted(components.keys()))}

print(component_id_map)

{'comp:build': 0, 'comp:gc': 1, 'comp:jclextensions': 2, 'comp:jvmti': 3, 'comp:test': 4, 'comp:vm': 5}


In [8]:
train_df = df_train[df_train["component"].isin(component_id_map.keys())]
test_df = df_train[df_train["component"].isin(component_id_map.keys())]

In [9]:
train_df["component_id"] = train_df["component"].apply(lambda x: component_id_map[x])
test_df["component_id"] = test_df["component"].apply(lambda x: component_id_map[x])

/tmp/ipykernel_1290870/771724263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["component_id"] = train_df["component"].apply(lambda x: component_id_map[x])
/tmp/ipykernel_1290870/771724263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["component_id"] = test_df["component"].apply(lambda x: component_id_map[x])


# Load Models

In [10]:
len(test_df.component_id.unique())

6

### Prepare Tokenizer

In [11]:
with open("/home/mdafifal.mamun/notebooks/triagerX/triagerx/config/config.yaml", "r") as file:
    configs = yaml.safe_load(file)

In [12]:
tokenizer_config = configs["tokenizer"]

In [18]:
developer_config["use_special_tokens"]

False

In [13]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_config["model_name"])

if developer_config["use_special_tokens"]:
    logger.debug("Using special tokens...")
    special_tokens = TextProcessor.SPECIAL_TOKENS
    special_tokens_dict = {"additional_special_tokens": list(special_tokens.values())}
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# Load Trained Models

In [14]:
def get_trained_model(weight_path: str, num_classes, unfrozen_layers, dropout, base_model, tokenizer):
    model = LBTPDeberta(
        output_size=num_classes, 
        unfrozen_layers=unfrozen_layers, 
        dropout=dropout, 
        base_model=base_model
    )

    model.base_model.resize_token_embeddings(len(tokenizer))

    model.load_state_dict(torch.load(weight_path))

    return model

In [15]:
component_model_weights =  os.path.join(
    component_config.get("weights_save_location"), f"{component_config['run_name']}_seed{seed}.pt"
)
developer_model_weights = os.path.join(
    developer_config.get("weights_save_location"), f"{developer_config['run_name']}_seed{seed}.pt"
)

In [16]:
len(test_df.component_id.unique())

6

In [17]:
component_model = get_trained_model(
    weight_path=component_model_weights,
    num_classes=len(test_df.component_id.unique()), 
    unfrozen_layers=component_config["unfrozen_layers"], 
    dropout=component_config["dropout"], 
    base_model=component_config["base_transformer_model"],
    tokenizer=tokenizer
)


RuntimeError: Error(s) in loading state_dict for LBTPDeberta:
	size mismatch for base_model.embeddings.word_embeddings.weight: copying a param with shape torch.Size([50273, 768]) from checkpoint, the shape in current model is torch.Size([50265, 768]).

In [ ]:
developer_model = get_trained_model(
    weight_path=developer_model_weights,
    num_classes=len(train_df.owner_id.unique()), 
    unfrozen_layers=developer_config["unfrozen_layers"], 
    dropout=developer_config["dropout"], 
    base_model=developer_config["base_transformer_model"],
    tokenizer=tokenizer
)

In [ ]:
similarity_model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
trx = TriagerX(
    developer_prediction_model=developer_model,
    component_prediction_model=component_model,
    similarity_model=similarity_model,
    tokenizer=tokenizer,
    tokenizer_config=tokenizer_config,
    train_data=train_df,
    developer_id_map=developer_id_map,
    component_id_map=component_id_map,
    component_developers_map=components,
    issues_path="/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/issue_data",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

2024-05-16 13:24:09.413 | DEBUG    | triagerx.system.triagerx:__init__:57 - Using device: cuda
2024-05-16 13:24:09.415 | DEBUG    | triagerx.system.triagerx:__init__:58 - Generating embedding for existing issues...


# Generate Recommendations

In [ ]:
def get_recommendation_for_test_index(test_df, test_idx):
    actual_owner = test_df.iloc[test_idx].owner
    actual_component = test_df.iloc[test_idx].component
    issue_url = test_df.iloc[test_idx].issue_url

    print("Issue:", test_df.iloc[test_idx].issue_url)
    print("Actual component:", actual_component)
    print("Actual owner:", actual_owner)
    recommendation = trx.get_recommendation(
        test_df.iloc[test_idx].text,
        k_comp=3,
        k_dev=3,
        k_rank=5,
        similarity_threshold=0.5
    )

    status = None

    if actual_owner in recommendation["predicted_developers"]:
        status = "Matched"
    else:
        status = "Not Matched"

    recommendation["actual_owner"] = actual_owner
    recommendation["issue_url"] = issue_url
    recommendation["actual_component"] = actual_component
    recommendation["model_pred_status"] = status
    recommendation["rank_status"] = "Matched" if actual_owner in recommendation["combined_ranking"][:3] else "Not Matched"

    return recommendation

In [ ]:
recommendations_matched = []
component_matched = []
ranked_matched = []

for idx in range(len(test_df)):
    rec = get_recommendation_for_test_index(test_df, idx)
    
    if rec["model_pred_status"] == "Matched":
        recommendations_matched.append(rec)
    
    if rec["actual_component"] in rec["predicted_components"]:
        component_matched.append(rec)

    if rec["rank_status"] == "Matched":
        ranked_matched.append(rec)

2024-05-16 13:24:17.624 | DEBUG    | triagerx.system.triagerx:get_recommendation:64 - Processing issue...
2024-05-16 13:24:17.628 | DEBUG    | triagerx.system.triagerx:get_recommendation:68 - Prediciting components...


Issue: https://github.com/eclipse-openj9/openj9/issues/54
Actual component: comp:vm
Actual owner: hangshao0


2024-05-16 13:24:17.940 | INFO     | triagerx.system.triagerx:get_recommendation:75 - Predicted components: ['comp:vm', 'comp:test', 'comp:gc']
2024-05-16 13:24:17.942 | DEBUG    | triagerx.system.triagerx:get_recommendation:77 - Generating developer recommendation...
2024-05-16 13:24:18.002 | INFO     | triagerx.system.triagerx:get_recommendation:84 - Recommended developers: ['hangshao0', 'babsingh', 'pshipton', 'gacholio', 'tajila']
2024-05-16 13:24:18.003 | DEBUG    | triagerx.system.triagerx:get_recommendation:86 - Generating recommendation by similarity...
2024-05-16 13:24:18.481 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:219 - Skipped users: {'DanHeidinga', 'manqingl', 'dhong44', 'smudigon', 'SueChaplain', 'rmnattas', 'thomaslittletrainx', 'pdbain-ibm', 'mgaudet', 'thjaeckle', 'mstoodle'} because they don't exist in the ${expected_developers} list
2024-05-16 13:24:18.483 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:179 - [('babsi

Issue: https://github.com/eclipse-openj9/openj9/issues/87
Actual component: comp:build
Actual owner: keithc-ca


2024-05-16 13:24:18.731 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:195 - Skipping issue as label id 2 did not match with any of [5, 0, 1]
2024-05-16 13:24:18.785 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:219 - Skipped users: {'DanHeidinga', 'andrew-m-leonard', 'M-Davies', 'JamesKingdon', 'jdekonin'} because they don't exist in the ${expected_developers} list
2024-05-16 13:24:18.787 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:179 - [('keithc-ca', 25.984186097979546), ('pshipton', 5.732307583093643), ('ChengJin01', 1.9238431751728058)]
2024-05-16 13:24:18.788 | INFO     | triagerx.system.triagerx:get_recommendation:95 - Recommended developers by issue similarity: ['keithc-ca', 'pshipton', 'ChengJin01']
2024-05-16 13:24:18.789 | DEBUG    | triagerx.system.triagerx:get_recommendation:97 - Aggregating ranking...
2024-05-16 13:24:18.791 | INFO     | triagerx.system.triagerx:get_recommendation:101 - Recommended devel

...

Issue: https://github.com/eclipse-openj9/openj9/issues/16439
Actual component: comp:vm
Actual owner: babsingh
Issue: https://github.com/eclipse-openj9/openj9/issues/16449
Actual component: comp:jclextensions
Actual owner: pshipton


2024-05-16 13:27:27.023 | INFO     | triagerx.system.triagerx:get_recommendation:84 - Recommended developers: ['pshipton', 'keithc-ca', 'babsingh', 'JasonFengJ9', 'thallium']
2024-05-16 13:27:27.025 | DEBUG    | triagerx.system.triagerx:get_recommendation:86 - Generating recommendation by similarity...
2024-05-16 13:27:27.118 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:195 - Skipping issue as label id 0 did not match with any of [2, 5, 1]
2024-05-16 13:27:27.124 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:219 - Skipped users: {'NagarajuGunda'} because they don't exist in the ${expected_developers} list
2024-05-16 13:27:27.125 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:179 - [('pshipton', 71.59411385655403), ('keithc-ca', 5.293283611536026), ('JasonFengJ9', 3.205174207687378), ('hangshao0', 0.771641805768013)]
2024-05-16 13:27:27.126 | INFO     | triagerx.system.triagerx:get_recommendation:95 - Recommended develo

Issue: https://github.com/eclipse-openj9/openj9/issues/16460
Actual component: comp:test
Actual owner: babsingh
Issue: https://github.com/eclipse-openj9/openj9/issues/16461
Actual component: comp:vm
Actual owner: babsingh


2024-05-16 13:27:27.358 | INFO     | triagerx.system.triagerx:get_recommendation:84 - Recommended developers: ['babsingh', 'JasonFengJ9', 'pshipton', 'tajila', 'llxia']
2024-05-16 13:27:27.359 | DEBUG    | triagerx.system.triagerx:get_recommendation:86 - Generating recommendation by similarity...
2024-05-16 13:27:27.385 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:179 - [('JasonFengJ9', 16.756311893463135), ('pshipton', 9.449226021766663), ('babsingh', 5.000000596046448), ('llxia', 4.493808299303055)]
2024-05-16 13:27:27.387 | INFO     | triagerx.system.triagerx:get_recommendation:95 - Recommended developers by issue similarity: ['JasonFengJ9', 'pshipton', 'babsingh', 'llxia']
2024-05-16 13:27:27.388 | DEBUG    | triagerx.system.triagerx:get_recommendation:97 - Aggregating ranking...
2024-05-16 13:27:27.389 | INFO     | triagerx.system.triagerx:get_recommendation:101 - Recommended developers by ranking aggregation: ['JasonFengJ9', 'babsingh', 'pshipton']
2024

Issue: https://github.com/eclipse-openj9/openj9/issues/16462
Actual component: comp:test
Actual owner: JasonFengJ9


In [ ]:
dev_model_score = len(recommendations_matched) / len(test_df)
ranked_score = len(ranked_matched) / len(test_df)
component_model_score = len(component_matched) / len(test_df)

print("Top 3 Recommendation Score")
print("=============================================")
print(
    f"Developer Recommendation Model: {dev_model_score}\n"
    f"Ranking Aggregation Score: {ranked_score}\n"
    f"Component Prediction Score: {component_model_score}\n"
)

Top 3 Recommendation Score
Developer Recommendation Model: 0.9368327402135231
Ranking Aggregation Score: 0.9670818505338078
Component Prediction Score: 0.9857651245551602



In [ ]:
print(train_df.iloc[790].text)

Bug Title: VT build fails to compile on Windows
Bug Description: https://openj9-jenkins.osuosl.org/job/Build_JDKnext_x86-64_windows_vt_standard_Personal/1/consoleFull

```
14:44:21  NativeHelpers.cpp
14:44:21  ObjectFieldInfo.cpp
14:44:22  [ 51%] Building CXX object runtime/vm/CMakeFiles/j9vm.dir/ObjectMonitor.cpp.obj
14:44:22  f:\users\jenkins\workspace\build_jdknext_x86-64_windows_vt_standard_personal\openj9\runtime\vm\ObjectFieldInfo.cpp(53): error C2220: the following warning is treated as an error
14:44:22  f:\users\jenkins\workspace\build_jdknext_x86-64_windows_vt_standard_personal\openj9\runtime\vm\ObjectFieldInfo.cpp(53): warning C4244: 'initializing': conversion from 'UDATA' to 'U_32', possible loss of data
14:44:22  f:\users\jenkins\workspace\build_jdknext_x86-64_windows_vt_standard_personal\openj9\runtime\vm\ObjectFieldInfo.cpp(77): warning C4319: '~': zero extending 'U_32' to 'UDATA' of greater size
14:44:22  make[6]: *** [runtime/vm/CMakeFiles/j9vm.dir/build.make:1618: run

In [ ]:
test_df.iloc[15]

Unnamed: 0                                                     80
issue_number                                                  204
issue_title     SunFlow benchmark: unusual sensitivity to gcpo...
description     The SunFlow benchmark performed poorly with th...
issue_url       https://github.com/eclipse-openj9/openj9/issue...
issue_state                                                  open
creator                                              JamesKingdon
labels                                        comp:gc, userRaised
owner                                                      amicic
component                                                 comp:gc
summary         Here is a summarized version of the bug report...
text            Bug Title: SunFlow benchmark: unusual sensitiv...
owner_id                                                        4
component_id                                                    1
Name: 80, dtype: object

In [ ]:
get_recommendation_for_test_index(test_df, 51)

2024-05-16 13:27:27.593 | DEBUG    | triagerx.system.triagerx:get_recommendation:64 - Processing issue...
2024-05-16 13:27:27.596 | DEBUG    | triagerx.system.triagerx:get_recommendation:68 - Prediciting components...
2024-05-16 13:27:27.623 | INFO     | triagerx.system.triagerx:get_recommendation:75 - Predicted components: ['comp:vm', 'comp:gc', 'comp:jvmti']
2024-05-16 13:27:27.625 | DEBUG    | triagerx.system.triagerx:get_recommendation:77 - Generating developer recommendation...
2024-05-16 13:27:27.684 | INFO     | triagerx.system.triagerx:get_recommendation:84 - Recommended developers: ['pshipton', 'JasonFengJ9', 'keithc-ca', 'gacholio', 'tajila']
2024-05-16 13:27:27.686 | DEBUG    | triagerx.system.triagerx:get_recommendation:86 - Generating recommendation by similarity...
2024-05-16 13:27:27.699 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:179 - [('pshipton', 10.000001192092896), ('gacholio', 1.250000149011612)]
2024-05-16 13:27:27.700 | INFO     | tri

Issue: https://github.com/eclipse-openj9/openj9/issues/1016
Actual component: comp:vm
Actual owner: pshipton


{'predicted_components': ['comp:vm', 'comp:gc', 'comp:jvmti'],
 'comp_prediction_score': [55.1951904296875,
  4.628835678100586,
  -9.205215454101562],
 'predicted_developers': ['pshipton', 'JasonFengJ9', 'keithc-ca'],
 'dev_prediction_score': [17.143863677978516,
  10.478222846984863,
  10.406988143920898,
  7.673142910003662,
  4.848223686218262],
 'similar_devs': [('pshipton', 10.000001192092896),
  ('gacholio', 1.250000149011612)],
 'combined_ranking': ['pshipton', 'JasonFengJ9', 'keithc-ca'],
 'actual_owner': 'pshipton',
 'issue_url': 'https://github.com/eclipse-openj9/openj9/issues/1016',
 'actual_component': 'comp:vm',
 'model_pred_status': 'Matched',
 'rank_status': 'Matched'}